In [1]:
import logging
import os
import sys
from collections import defaultdict
import yaml
import json
import os
import re
import logging
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from functools import partial
from rdflib import Graph, Literal, URIRef
from rdflib.namespace import RDF, RDFS
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import dycomutils as common_utils
from typing import List, Dict, Any, Optional, Set, Tuple, DefaultDict
import openai

sys.path.append("/home/desild/work/research/chatbs/v2")

from src.utils.helpers import setup_logger
from src.utils.parser import graph_query_to_sexpr, is_inv_rel, get_inv_rel, graph_query_to_sparql
from src.utils.kg import get_readable_relation, get_readable_class, get_non_literals, get_nodes_by_class, \
    get_reverse_relation, get_reverse_readable_relation, prune_graph_query, legal_class, legal_relation
from src.utils.arguments import Arguments
from src.utils.sparql import SPARQLUtil, get_freebase_label, get_freebase_literals_by_cls_rel, \
    get_freebase_entid_lbl_by_cls
from src.utils.maps import literal_map

from transformers import set_seed
from tqdm import tqdm

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

from src.explorer_updates import Explorer, ExecutableProgram
from src.utils.graph_manager import GraphManager

11/25/2025 14:31:51 - INFO - src.explorer_updates -   Loading .env file from: /home/desild/work/research/chatbs/v2/.env
11/25/2025 14:31:51 - INFO - src.explorer_updates -   Loading config: /home/desild/work/research/chatbs/v2/prov.config.yaml
11/25/2025 14:31:51 - INFO - src.explorer_updates -   Loading metadata: /home/desild/work/research/chatbs/v2/data/workflow/10_sample_graph/chatbs_sample_metadata.json
11/25/2025 14:31:51 - INFO - src.explorer_updates -   Initializing GraphManager...
11/25/2025 14:31:51 - INFO - src.explorer_updates -   Graph loaded with 24073 triples.


In [2]:
def llm_chat(system_prompt: str, user_prompt: str, model_version: str, structured_output: bool = False) -> str:
    """
    Sends a chat request to an OpenAI-compatible API.
    R: llm_chat
    """
    client = None
    # R: if ((startsWith(model_version, "gpt-")) || (startsWith(model_version, "o1-")))
    if model_version.startswith("gpt-") or model_version.startswith("o1-"):
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("OPENAI_API_KEY not set in .env file")
        client = openai.OpenAI(api_key=api_key)
    else:
        # R: base_url = "http://idea-llm-01.idea.rpi.edu:5000/v1/"
        client = openai.OpenAI(
            base_url="http://idea-llm-01.idea.rpi.edu:5000/v1/",
            api_key=os.getenv("LOCAL_LLM_API_KEY", "no-key-needed") # Add your local key to .env if needed
        )
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    chat_params = {
        "model": model_version,
        "messages": messages
    }
    
    # R: if (!is.null(structured_output))
    if structured_output:
        log.info("Requesting structured (JSON) output from LLM.")
        # This is the modern way to request JSON from OpenAI
        chat_params["response_format"] = {"type": "json_object"}
        
    try:
        response = client.chat.completions.create(**chat_params)
        answer = response.choices[0].message.content
        return answer
    except Exception as e:
        log.error(f"Error in LLM chat: {e}")
        return f"Error: {e}"

def update_answer(system_prompt: str, user_prompt: str, generated_answer: str, error_message: str, model_version: str) -> str:
    """
    Asks the LLM to correct a previous, failed response.
    R: update_answer
    """
    recorrection_template = f"""
    User prompt : {user_prompt}
    Incorrect generated answer : {generated_answer}
    Error message : {error_message}
    Analyze the original user prompt, the incorrect answer, and the error message. Identify where the generated response failed to meet the prompt’s intent. Then, provide a revised answer.
    """
    return llm_chat(system_prompt, recorrection_template, model_version)


def create_timestamp_id(prefix:str):
    """
    Creates a unique identifier based on the current timestamp.
    R: create_timestamp_id
    """
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    return f"{prefix}_{timestamp}"

In [3]:
# --- 1. Setup & Configuration ---
ROOT_DIR = os.path.abspath("/home/desild/work/research/chatbs")
V2_DIR = os.path.join(ROOT_DIR, "v2")
EXPLORED_PROGRAMS_PICKLE = "data/workflow/explored_programs.pkl"

# Setup basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = logging.getLogger(__name__)

In [4]:
# Load .env file from the specified path
# R: load_dot_env("../ChatBS-NexGen/.env")
env_path = os.path.join(V2_DIR, ".env")
log.info(f"Loading .env file from: {env_path}")
load_dotenv(env_path)

# Load YAML config
# R: config <- yaml::read_yaml(...)
config_path = os.path.join(V2_DIR, "prov.config.yaml")
log.info(f"Loading config: {config_path}")
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Load JSON metadata
# R: ttl_metadata <- readLines("QGraph_metadata.json")
metadata_path = os.path.join(V2_DIR, "data/workflow/chatbs_sample_metadata.json")
log.info(f"Loading metadata: {metadata_path}")
with open(metadata_path, 'r') as f:
    ttl_metadata = json.load(f)

11/25/2025 14:31:51 - INFO - __main__ -   Loading .env file from: /home/desild/work/research/chatbs/v2/.env
11/25/2025 14:31:51 - INFO - __main__ -   Loading config: /home/desild/work/research/chatbs/v2/prov.config.yaml
11/25/2025 14:31:51 - INFO - __main__ -   Loading metadata: /home/desild/work/research/chatbs/v2/data/workflow/chatbs_sample_metadata.json


In [5]:
graph_manager = GraphManager(config, os.path.join(V2_DIR, "data/workflow/10_sample_graph/chatbs_sample.ttl"))
schema = common_utils.serialization.load_json(os.path.join(V2_DIR, "data/workflow/schema.json"))
definitions = {'class_definitions':schema['classes'], 'relation_definitions':{k:v["description"] for k,v in schema['relations'].items()}}

11/25/2025 14:31:51 - INFO - src.utils.graph_manager -   Initializing GraphManager...
11/25/2025 14:31:51 - INFO - src.utils.graph_manager -   Graph loaded with 4062 triples.


In [6]:
def path_to_question(exp:ExecutableProgram, exp1:ExecutableProgram, schema:Dict) -> str:
    exp1_q_path = exp1.metadata['path']
    exp1_ans = exp1.example_output.head(2)
    exp1_query = exp1.code
    exp1_question = "What was the program that generated this data item?"
    
    """
    Converts a single path (a list of node URIs) into a question-answering format utilizing an 
    LLM.
    """
    exp1 = """
    ### sparql query
    {exp_q}
    ### sparql result
    {exp_ans}
    ### path
    {path}
    ### question
    {question}
    """.format(
        path=exp1_q_path, 
        question=exp1_question,
        exp_q=exp1_query,
        exp_ans=exp1_ans.to_dict('list')
    )
    
    PROMPT_TEMPLATE = """
    Given the following SPARQL query and its result and the path it represents in the RDF graph, 
    generate a natural language question that would lead to this query and result.
    
    The symbolic definitions and an example are provided below.
    ##Definitions:
    {schema}
    
    ##Example1
    {exp1}
    
    ### sparql query
    {exp_q}
    ### sparql result
    {exp_ans}
    ### path
    {path}
    """
    
    return PROMPT_TEMPLATE.format(
        exp1=exp1, 
        schema=json.dumps(schema, indent=2),
        path=exp.metadata['path'], 
        exp_q=exp.code,
        exp_ans=exp.example_output.head(2).to_dict('list')
    )

In [7]:
explorations = common_utils.serialization.load_pickle(os.path.join(V2_DIR, EXPLORED_PROGRAMS_PICKLE))

In [8]:
explorations = {' -> '.join(exp.metadata['path']): exp for exp in explorations if exp.metadata}

In [9]:
for k in explorations.keys():
    print(f"Exploration Key: {k}")

Exploration Key: provone:Program -> provone:hasOutPort -> provone:Port
Exploration Key: provone:Program -> provone:hasInPort -> provone:Port
Exploration Key: provone:Program -> provone:hasOutPort -> provone:Port -> provone:connectsTo -> provone:Channel
Exploration Key: provone:Program -> provone:hasInPort -> provone:Port -> provone:connectsTo -> provone:Channel
Exploration Key: prov:Usage -> provone:hadInPort -> provone:Port
Exploration Key: prov:Usage -> provone:hadEntity -> provone:Data
Exploration Key: prov:Usage -> provone:hadInPort -> provone:Port -> provone:connectsTo -> provone:Channel
Exploration Key: prov:Usage -> provone:hadEntity -> provone:Data -> prov:wasGeneratedBy -> provone:Execution
Exploration Key: prov:Usage -> provone:hadEntity -> provone:Data -> prov:wasGeneratedBy -> provone:Execution -> prov:qualifiedAssociation -> prov:Association
Exploration Key: prov:Usage -> provone:hadEntity -> provone:Data -> prov:wasGeneratedBy -> provone:Execution -> prov:qualifiedGenerat

In [10]:
exp1 = explorations['provone:Data -> prov:wasGeneratedBy -> provone:Execution -> prov:qualifiedAssociation -> prov:Association -> prov:hadPlan -> provone:Program']

In [11]:

def add_parameter(main_id:str, inp_n, inp, dtype="xsd:string", required="true") -> Dict:
    """ ex:intParameterA
        a             fno:Parameter ;
        fno:predicate ex:startValue ;
        fno:type      xsd:integer ;
        fno:required  "true"^^xsd:boolean .
        
    """   
    inp_id = create_timestamp_id(main_id + "_param")
    graph_manager.add_to_graph(
        inp_id,
        "a",
        "fno:Parameter"
    )
    graph_manager.add_to_graph(
        inp_id,
        "fno:predicate",
        inp_n,
        literal=True,
        dtype="xsd:string"
    )
    
    graph_manager.add_to_graph(
        inp_id,
        "rdfs:label",
        inp,
        literal=True,
        dtype="xsd:string"
    )
    graph_manager.add_to_graph(
        inp_id,
        "fno:type",
        dtype
    )
    graph_manager.add_to_graph(
        inp_id,
        "fno:required",
        required,
        literal=True,
        dtype="xsd:boolean"
    )
    
    return {"id": inp_id, "predicate": inp, "dtype": dtype, "required": required}

def add_return(main_id:str, out_n, out, dtype="xsd:string", required="true") -> Dict:
    """ 
    
    ex:sumOutput
    a             fno:Output ;
    fno:predicate ex:sumResult ;
    fno:type      xsd:integer ;
    fno:required  "true"^^xsd:boolean .
        
    """   
    out_id = create_timestamp_id(main_id + "_output")
    graph_manager.add_to_graph(
        out_id,
        "a",
        "fno:Output"
    )
    graph_manager.add_to_graph(
        out_id,
        "fno:predicate",
        out_n,
        literal=False,
        dtype="xsd:string"
    )
    
    graph_manager.add_to_graph(
        out_id,
        "rdfs:label",
        out,
        literal=False,
        dtype="xsd:string"
    )
    graph_manager.add_to_graph(
        out_id,
        "fno:type",
        dtype
    )
    graph_manager.add_to_graph(
        out_id,
        "fno:required",
        required,
        literal=True,
        dtype="xsd:boolean"
    )

    return {"id": out_id, "predicate": inp, "dtype": dtype, "required": required}

def add_problem(main_id:str, question:str, broader:Optional[str]=None) -> Dict:
    """ 
    ex:sumProblem
    a                   fno:Problem ;
    fno:name            "The sum problem"^^xsd:string ;
    dcterms:description "This handles the problem of adding two integers to each other."^^xsd:string ;
    skos:broader        ex:mathProblem .
    """
    
    problem_id = create_timestamp_id(main_id + "_problem")
    graph_manager.add_to_graph(
        problem_id,
        "a",
        "fno:Problem"
    )
    graph_manager.add_to_graph(
        problem_id,
        "fno:name",
        question,
        literal=True,
        dtype="xsd:string"
    )
    
    if broader:
        graph_manager.add_to_graph(
            problem_id,
            "skos:broader",
            broader,
            literal=False
        )

    return {"id": problem_id, "name": question, "broader": broader}

def add_function(main_id:str, name:str, question_id:str, parameters:List[Dict], returns:List[Dict]) -> Dict:
    """
    ex:sumFunction
    a                   fno:Function ;
    fno:name            "The sum function"^^xsd:string ;
    dcterms:description "This function can do the sum of two integers."^^xsd:string ;
    fno:solves          ex:sumProblem ;
    fno:implements      ex:sumAlgorithm ;
    fno:expects         ( ex: intParameterA ex:intParameterB ) ;
    fno:returns         ( ex:sumOutput ) .
    """
    
    function_id = create_timestamp_id(main_id + "_function")
    graph_manager.add_to_graph(
        function_id,
        "a",
        "fno:Function"
    )
    graph_manager.add_to_graph(
        function_id,
        "fno:name",
        name,
        literal=True,
        dtype="xsd:string"
    )
    
    graph_manager.add_to_graph(
        function_id,
        "fno:solves",
        question_id,
        literal=False
    )
    
    for param in parameters:
        graph_manager.add_to_graph(
            function_id,
            "fno:expects",
            param['id'],
            literal=False
        )
        
    for ret in returns:
        graph_manager.add_to_graph(
            function_id,
            "fno:returns",
            ret['id'],
            literal=False
        )

    return {"id": function_id, "name": name, "solves": question_id, "parameters": parameters, "returns": returns}
    
def add_implementation(main_id:str, code:str) -> Dict:
    """
    ex:leftPadImplementation
    a         fnoi:NpmPackage ;
    doap:name "left-pad" .
    
    """
    
    function_id = create_timestamp_id(main_id + "_function")
    graph_manager.add_to_graph(
        function_id,
        "a",
        "fnoi:SQLImplementation"
    )
    
    graph_manager.add_to_graph(
        function_id,
        "rdfs:label",
        code,
        literal=True,
        dtype="xsd:string"
    )
    
    return {"id":function_id, "label":code}

def add_mapping(main_id:str, function_id:str, implementation_id:str, parameters:List[Dict], returns:List[Dict]):
    """
    ex:leftPadMapping
    a                    fno:Mapping ;
    fno:function         ex:leftPad ;
    fno:implementation   ex:leftPadImplementation ;
    fno:methodMapping    [ a                fnom:StringMethodMapping ;
                           fnom:method-name "doLeftPadding" ] ;
    fno:parameterMapping [ a                                    fnom:PositionParameterMapping ;
                           fnom:functionParameter               ex:inputStringParameter ;
                           fnom:implementationParameterPosition "2"^^xsd:int ] ;
    fno:parameterMapping [ a                                    fnom:PositionParameterMapping ;
                           fnom:functionParameter               ex:paddingParameter ;
                           fnom:implementationParameterPosition "1"^^xsd:int ] ;
    fno:returnMapping    [ a                   fnom:DefaultReturnMapping ;
                           fnom:functionOutput ex:outputStringOutput ] .
    """
    
    mapping_id = create_timestamp_id(main_id + "_mapping")
    graph_manager.add_to_graph(
        mapping_id,
        "a",
        "fno:Mapping"
    )
    
    graph_manager.add_to_graph(
        mapping_id,
        "fno:function",
        function_id
    )

    graph_manager.add_to_graph(
        mapping_id,
        "fno:implementation",
        implementation_id
    )

    for i, param in enumerate(parameters):
        _id = create_timestamp_id(main_id + "_parameter_mapping_" + str(i))
        graph_manager.add_to_graph(
            mapping_id,
            "fno:parameterMapping",
            _id,
            literal=False
        )

        graph_manager.add_to_graph(
            _id,
            "a",
            "fnom:PositionParameterMapping"
        )
        graph_manager.add_to_graph(
            _id,
            "fnom:functionParameter",
            param['id'],
            literal=False
        )
        graph_manager.add_to_graph(
            _id,
            "fnom:implementationParameterPosition",
            str(i),
            literal=True,
            dtype="xsd:int"
        )

    for i, ret in enumerate(returns):
        _id = create_timestamp_id(main_id + "_return_mapping_" + str(i))
        graph_manager.add_to_graph(
            mapping_id,
            "fno:returnMapping",
            _id,
            literal=False
        )

        graph_manager.add_to_graph(
            _id,
            "a",
            "fnom:DefaultReturnMapping"
        )
        graph_manager.add_to_graph(
            _id,
            "fnom:functionOutput",
            ret['id'],
            literal=False
        )
        
def add_example(main_id:str, function_id:str, example_usage:str, example_output:Dict):
    execution_id = create_timestamp_id(main_id + "_executes")
    graph_manager.add_to_graph(
        execution_id,
        "a",
        "fno:Execution"
    )
    
    graph_manager.add_to_graph(
        function_id,
        "fno:executes",
        execution_id
    )
    
    graph_manager.add_to_graph(
        execution_id,
        "rdfs:label",
        example_usage,
        literal=True,
        dtype="xsd:string"
    )
    
    return {"id": execution_id, "label": example_usage, "output": example_output}

    
    

def add_fno_graph(index:int, exp:ExecutableProgram, question:str, categories:str):
    """
    Adds an FNO graph representation to the given ExecutableProgram.
    R: add_fno_graph
    """
    exp.program_id = "ques:eprog{}".format(index)
    

    param_input, param_output = [], []
    for k,v in exp.input_spec.items():
        param_input.append(add_parameter(exp.program_id+f"_{k}", k, v))
        
    for k,v in exp.output_spec.items():
        param_output.append(add_parameter(exp.program_id+f"_{k}", k, v))
        
    question_id = add_problem(
        exp.program_id, 
        question)
        
    function = add_function(
        exp.program_id, 
        categories,
        question_id['id'], 
        param_input, 
        param_output
        )
    
    implementation = add_implementation(
        exp.program_id,
        exp.code
    )

    example = add_example(
        exp.program_id,
        function['id'],
        exp.example_usage,
        exp.example_output.to_dict(orient='list')
    )
    
    

    #parameters
    add_mapping(
            exp.program_id, 
            function['id'], 
            implementation['id'],
            param_input, 
            param_output
    )
    
    

In [12]:
    
for i, (k,v) in tqdm(enumerate(explorations.items()), total=len(explorations), desc="Processing Explorations"):
    print(f"Exploration Key: {k}")
    
    prompt = path_to_question(v, exp1, definitions)
    #print(prompt)
    
    question = llm_chat(
        prompt,
        "what is the question?",
        "gpt-4o"
    )
    
    #question = "test question"

    add_fno_graph(i, v, question, "->".join(v.metadata['path']))

Processing Explorations:   0%|          | 0/80 [00:00<?, ?it/s]

Exploration Key: provone:Program -> provone:hasOutPort -> provone:Port


11/25/2025 14:31:53 - INFO - httpx -   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing Explorations:   1%|▏         | 1/80 [00:01<02:00,  1.52s/it]

Exploration Key: provone:Program -> provone:hasInPort -> provone:Port


11/25/2025 14:31:55 - INFO - httpx -   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing Explorations:   2%|▎         | 2/80 [00:03<02:02,  1.57s/it]

...

11/25/2025 14:33:25 - INFO - httpx -   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing Explorations:  98%|█████████▊| 78/80 [01:33<00:02,  1.25s/it]

Exploration Key: provone:Data -> prov:wasGeneratedBy -> provone:Execution -> prov:qualifiedAssociation -> prov:Association -> prov:hadPlan -> provone:Program -> provone:hasOutPort -> provone:Port -> provone:connectsTo -> provone:Channel


11/25/2025 14:33:26 - INFO - httpx -   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing Explorations:  99%|█████████▉| 79/80 [01:34<00:01,  1.22s/it]

Exploration Key: provone:Data -> prov:wasGeneratedBy -> provone:Execution -> prov:qualifiedAssociation -> prov:Association -> prov:hadPlan -> provone:Program -> provone:hasInPort -> provone:Port -> provone:connectsTo -> provone:Channel


11/25/2025 14:33:28 - INFO - httpx -   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing Explorations: 100%|██████████| 80/80 [01:35<00:00,  1.20s/it]


In [13]:
def extract_categories(path:str) -> Tuple[str, str]:
    """
    Extracts categories from a given path.
    R: extract_categories
    """
    
    _split = path.split("|")
    fst = _split[0].strip().split(" ")[-1].strip()
    snd = ":".join(_split[2].strip().split(" ")[0].strip().split(":")[1:])
    return f"{fst} -> {snd}", fst


In [14]:
explorations = common_utils.serialization.load_pickle(os.path.join(V2_DIR, EXPLORED_PROGRAMS_PICKLE))
explorations = {exp.program_id: exp for exp in explorations if not exp.metadata}
cls_set = set()

for i, (k,v) in tqdm(enumerate(explorations.items()), total=len(explorations), desc="Processing Explorations"):
    if "Explores objects of a given class in the RDF graph." not in v.description and "Explores Attributes of a given" not in v.description:
        print(f"Exploration Key: {k} - {v.description}")
        
        categories, cls = extract_categories(v.program_id)
        #add_fno_graph(i, v, v.description, categories)
        cls_set.add(cls)

    else:
        print(f"Skipping Exploration Key: {k}")

Processing Explorations: 100%|██████████| 38/38 [00:00<00:00, 123744.99it/s]

Skipping Exploration Key: explore_object_of_class
Skipping Exploration Key: explore_attr_of_object
Exploration Key: explore_object_of_class provone:Program | find by object uri value | relation:dc:description - For a given object with uri of class provone:Program, find all dc:description values.
Exploration Key: explore_object_of_class provone:Program | find by prop value | relation:dc:description - For a given prop value of class provone:Program, find all dc:description prop value of the object.
Exploration Key: explore_object_of_class provone:Program | find by object uri value | relation:dcterms:identifier - For a given object with uri of class provone:Program, find all dcterms:identifier values.
Exploration Key: explore_object_of_class provone:Program | find by prop value | relation:dcterms:identifier - For a given prop value of class provone:Program, find all dcterms:identifier prop value of the object.
Exploration Key: explore_object_of_class provone:Program | find by object uri v

In [15]:
for i, (k,v) in tqdm(enumerate(explorations.items()), total=len(explorations), desc="Processing Explorations"):
    if "Explores objects of a given class in the RDF graph." in v.description:
        add_fno_graph(i, v, "Explores objects of a given class in the RDF graph.", " -> ".join(list(cls_set)))
        
    if "Explores Attributes of a given" in v.description:
        add_fno_graph(i, v, "Explores Attributes of a given object in the RDF graph.", " -> ".join(list(cls_set)))

Processing Explorations: 100%|██████████| 38/38 [00:00<00:00, 38285.74it/s]


In [16]:
" -> ".join(list(cls_set))

'provone:Channel -> provone:Port -> provone:Program -> provone:Data -> provone:Execution'

In [17]:
graph_manager.save_graph(os.path.join(V2_DIR, "data/workflow/explored_programs_fno.ttl"))

11/25/2025 14:33:28 - INFO - src.utils.graph_manager -   Graph saved to /home/desild/work/research/chatbs/v2/data/workflow/explored_programs_fno.ttl


In [18]:
sparql_query2 = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
        PREFIX eo: <https://purl.org/heals/eo#>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX dc: <http://purl.org/dc/elements/1.1/>
        PREFIX food: <http://purl.org/heals/food/>
        PREFIX prov: <http://www.w3.org/ns/prov#>
        PREFIX provone: <http://purl.org/provone#>
        PREFIX sio:<http://semanticscience.org/resource/>
        PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
        PREFIX dcterms: <http://purl.org/dc/terms#>
        PREFIX user: <http://testwebsite/testUser#>
        PREFIX DFColumn: <http://testwebsite/testDFColumn#>
        PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
        PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX dbt: <http://dbpedia.org/resource/Template:>
        PREFIX ques: <http://atomic_questions.org/>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX fno: <https://w3id.org/function/vocabulary/core#>

        SELECT ?question_lbl ?function_name ?example ?param_map ?param_desc
        WHERE {
            ?map fno:implementation/rdfs:label ?imp ;
                       fno:function ?function ;
                       fno:parameterMapping/fnom:functionParameter/fno:predicate ?param_map ;
                       fno:parameterMapping/fnom:functionParameter/rdfs:label ?param_desc ;
                       fno:returnMapping/fnom:functionOutput/fno:predicate ?return_map ;
                       fno:returnMapping/fnom:functionOutput/rdfs:label ?return_desc .
            ?function fno:executes/rdfs:label ?example.
            ?function fno:solves/fno:name ?question_lbl .
            ?function fno:name ?function_name .
        }
"""

In [19]:
df = graph_manager.query(
    sparql_query2
)

In [20]:
df.to_csv("QUESTIONS_EXTRACTED_FROM_FNO_MAPPINGS.csv", index=False)

In [21]:
sparql_query2 = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
        PREFIX eo: <https://purl.org/heals/eo#>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX dc: <http://purl.org/dc/elements/1.1/>
        PREFIX food: <http://purl.org/heals/food/>
        PREFIX prov: <http://www.w3.org/ns/prov#>
        PREFIX provone: <http://purl.org/provone#>
        PREFIX sio:<http://semanticscience.org/resource/>
        PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
        PREFIX dcterms: <http://purl.org/dc/terms#>
        PREFIX user: <http://testwebsite/testUser#>
        PREFIX DFColumn: <http://testwebsite/testDFColumn#>
        PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
        PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
        PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        PREFIX dbt: <http://dbpedia.org/resource/Template:>
        PREFIX ques: <http://atomic_questions.org/>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        PREFIX fno: <https://w3id.org/function/vocabulary/core#>

        SELECT ?map ?imp ?question_lbl ?example ?param_map ?param_desc ?return_map ?return_desc
        WHERE {
            ?map fno:implementation/rdfs:label ?imp ;
                       fno:function ?function ;
                       fno:parameterMapping/fnom:functionParameter/fno:predicate ?param_map ;
                       fno:parameterMapping/fnom:functionParameter/rdfs:label ?param_desc ;
                       fno:returnMapping/fnom:functionOutput/fno:predicate ?return_map ;
                       fno:returnMapping/fnom:functionOutput/rdfs:label ?return_desc .
            ?function fno:executes/rdfs:label ?example.
            ?function fno:solves/fno:name ?question_lbl .
        }
"""

sparql_query1 = """

PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX ep: <http://linkedu.eu/dedalo/explanationPattern.owl#>
PREFIX eo: <https://purl.org/heals/eo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX food: <http://purl.org/heals/food/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX provone: <http://purl.org/provone#>
PREFIX sio:<http://semanticscience.org/resource/>
PREFIX cwfo: <http://cwf.tw.rpi.edu/vocab#>
PREFIX dcterms: <http://purl.org/dc/terms#>
PREFIX user: <http://testwebsite/testUser#>
PREFIX DFColumn: <http://testwebsite/testDFColumn#>
PREFIX fnom: <https://w3id.org/function/vocabulary/mapping#>
PREFIX fnoi: <hhttps://w3id.org/function/vocabulary/implementation#>
PREFIX fnoc: <https://w3id.org/function/vocabulary/composition/0.1.0/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbt: <http://dbpedia.org/resource/Template:>
PREFIX ques: <http://atomic_questions.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX fno: <https://w3id.org/function/vocabulary/core#>
    
SELECT ?mapping ?question_lbl ?paths
WHERE {
    ?mapping a fno:Mapping .
    ?mapping fno:function ?function .
    ?function fno:solves/fno:name ?question_lbl .
    ?function fno:name ?paths .
}


"""



In [22]:
df = graph_manager.query(
    sparql_query1
)

In [23]:
df

,mapping,question_lbl,paths
0,http://atomic_questions.org/eprog0_mapping_202...,What is the output port used by this program?,provone:Program->provone:hasOutPort->provone:Port
1,http://atomic_questions.org/eprog1_mapping_202...,What are the input ports for this program?,provone:Program->provone:hasInPort->provone:Port
2,http://atomic_questions.org/eprog2_mapping_202...,What is the channel that the output port of th...,provone:Program->provone:hasOutPort->provone:P...
3,http://atomic_questions.org/eprog3_mapping_202...,What is the channel connected to the input por...,provone:Program->provone:hasInPort->provone:Po...
4,http://atomic_questions.org/eprog4_mapping_202...,What is the input port used in this usage event?,prov:Usage->provone:hadInPort->provone:Port
...,...,...,...
77,http://atomic_questions.org/eprog77_mapping_20...,What is the channel that the input port for us...,provone:Data->prov:wasGeneratedBy->provone:Exe...
78,http://atomic_questions.org/eprog78_mapping_20...,What is the channel connected to the output po...,provone:Data->prov:wasGeneratedBy->provone:Exe...
79,http://atomic_questions.org/eprog79_mapping_20...,Through which channel is the input provided to...,provone:Data->prov:wasGeneratedBy->provone:Exe...
80,http://atomic_questions.org/eprog0_mapping_202...,Explores objects of a given class in the RDF g...,provone:Channel -> provone:Port -> provone:Pro...


In [24]:
df.to_csv(os.path.join(V2_DIR, "data/workflow/explored_programs_fno_qa.csv"), index=False)

In [25]:
from src.utils.graph_manager import GraphManager, regex_add_strings

df = graph_manager.query(
    regex_add_strings(sparql_query2, map_id=map_ids[0])
)

NameError: name 'map_ids' is not defined

In [ ]:
df[["imp", "question_lbl", "example", "param_map", "param_desc", "return_map", "return_desc"]].to_csv(os.path.join(V2_DIR, "data/workflow/explored_programs_fno_qa_example.csv"), index=False)